<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Android figurines and how to put the model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install and import the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

Import the required packages.

In [1]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/home/mjanuadi/anaconda3/envs/image-dl/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/mjanuadi/anaconda3/envs/image-dl/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do

### Prepare the dataset

This dataset contains about 70 images of 2 type of Android figurines: an Android and an Android pig. This is an example image from the dataset.

![android_figurine_sample.jpg](https://storage.googleapis.com/download.tensorflow.org/example_images/android_figurine_sample.jpg)

We start with downloading the dataset.

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [2]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/home/mjanuadi/ObjectDetection/data/train',
    '/home/mjanuadi/ObjectDetection/data/train',
    ['android', 'pig_android']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/home/mjanuadi/ObjectDetection/data/validate',
    '/home/mjanuadi/ObjectDetection/data/validate',
    ['android', 'pig_android']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite2 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [9]:
spec = model_spec.get('efficientdet_lite2')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [10]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
15/15 [==============================] - 36s 710ms/step - det_loss: 1.7052 - cls_loss: 1.1357 - box_loss: 0.0114 - reg_l2_loss: 0.0759 - loss: 1.7811 - learning_rate: 0.0065 - gradient_norm: 1.9117 - val_det_loss: 1.5254 - val_cls_loss: 1.0837 - val_box_loss: 0.0088 - val_reg_l2_loss: 0.0759 - val_loss: 1.6013
Epoch 2/20
15/15 [==============================] - 10s 650ms/step - det_loss: 1.4417 - cls_loss: 1.0384 - box_loss: 0.0081 - reg_l2_loss: 0.0759 - loss: 1.5176 - learning_rate: 0.0049 - gradient_norm: 2.8482 - val_det_loss: 1.2784 - val_cls_loss: 0.8999 - val_box_loss: 0.0076 - val_reg_l2_loss: 0.0759 - val_loss: 1.3543
Epoch 3/20
15/15 [==============================] - 7s 504ms/step - det_loss: 1.1840 - cls_loss: 0.8436 - box_loss: 0.0068 - reg_l2_loss: 0.0759 - loss: 1.2599 - learning_rate: 0.0048 - gradient_norm: 3.6448 - val_det_loss: 0.9682 - val_cls_loss: 0.6518 - val_box_loss: 0.0063 - val_reg_l2_loss: 0.0759 - val_loss: 1.0441
Epoch 4/20
15/15 [==============

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [11]:
model.evaluate(val_data)

1/1 [==============================] - 5s 5s/step



{'AP': 0.8732703,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.8732703,
 'ARmax1': 0.8875,
 'ARmax10': 0.8875,
 'ARmax100': 0.8875,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.8875,
 'AP_/android': 0.86110324,
 'AP_/pig_android': 0.8854373}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [12]:
model.export(export_dir='.', tflite_filename='android.tflite')

2024-04-11 11:37:18.754915: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 6.066 G  ops, equivalently 3.033 G  MACs


2024-04-11 11:37:26.258604: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2024-04-11 11:37:26.258674: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2024-04-11 11:37:26.258973: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpoye1eii0
2024-04-11 11:37:26.335696: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2024-04-11 11:37:26.335734: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpoye1eii0
2024-04-11 11:37:26.619057: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-04-11 11:37:29.178144: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpoye1eii0
2024-04-11 11:37:30.225242: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 3966285

Estimated count of arithmetic ops: 6.066 G  ops, equivalently 3.033 G  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0
2024-04-11 11:38:01.816463: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1963] Estimated count of arithmetic ops: 6.066 G  ops, equivalently 3.033 G  MACs



### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [13]:
model.evaluate_tflite('android.tflite', val_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


10/10 [==============================] - 40s 4s/step



{'AP': 0.8498762,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.8498762,
 'ARmax1': 0.85833335,
 'ARmax10': 0.85833335,
 'ARmax100': 0.85833335,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.85833335,
 'AP_/android': 0.8379538,
 'AP_/pig_android': 0.8617987}